In [574]:
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import sys
import traceback
from os import makedirs
from os.path import exists


def center(width, n):
    """
    Computes free space on the figure on both sides.
    :param width:
    :param n: number of algorithms
    :return:
    """
    max_unit = 1
    free_space = width - n * max_unit
    free_space = max(0.0, free_space / max_unit)
    free_left = free_space / 2
    free_right = free_space / 2
    return free_left, free_right
    

def diagram(list_of_algorithms, critical_distance, the_algorithm_candidate, output_figure_file, draw_cd_line,
            show_cd_text, draw_stats):
    """
    Draws critical distance diagram for Nemenyi or Bonferroni-Dunn post-hoc test.
    The diagram is shown if output_figure_file is None, and saved otherwise
    to the file.

    :param list_of_algorithms: [[(alg_name1, avg_rank1)], ...]
    :param critical_distance:
    :param output_figure_file: If not none, the diagram produced is saved to the specified file.
    Otherwise, the diagram is shown.
    :param the_algorithm_candidate: If we were performing Bonferroni-Dunn post-hcc test (1 vs all),
    this is the algorithm  from the list list_of_algorithms, which the other algorithms are compared to.
    If we were performing Nemenyi post-hoc test (all vs all), this should be None.
    :param draw_cd_line: boolean: whether to draw the critical distance line under the diagram or not
    :param show_cd_text: boolean: whether to show the critical distance or not
    :param draw_stats: boolean: whether to draw anything in addition to average ranks
    :return: output_figure_file
    """

    n = len(list_of_algorithms)
    sorted_algorithms = sorted(list_of_algorithms, key=lambda t: t[1])
    the_index = None
    if the_algorithm_candidate is not None:
        for i, alg_description in enumerate(sorted_algorithms):
            if alg_description[0] == the_algorithm_candidate:
                the_index = i
                break
        if the_index is None:
            print("{} not found among the results. We will draw Nemenyi style diagram.".format(the_algorithm_candidate))
    inf = float("inf")
    deltas = [inf] + [sorted_algorithms[i + 1][1] - sorted_algorithms[i][1] for i in range(n - 1)] + [inf]
    sorted_algos_copy = sorted(list_of_algorithms, key=lambda t: t[1])
    sorted_algos_copy = sorted_algos_copy[: n//2] + sorted_algos_copy[n//2:][::-1]  # for easier drawing
    # some plot parameters:
    inter_lines_space = 0.34
    link_length_bonus = 0.04
    names_lines_space = 0.12
    first_level_height = 0.1
    critical_distance_offset = -0.88    # position of the critical distance under the main plot
    end_of_line_manipulator = 1.8        # shorten the horizontal part of the line by that much
    font_size = 8
    # latex fonts
    font_properties = {'family': 'serif', 'serif': ['Computer Modern Roman'],
                      'weight': 'normal', 'size': font_size}
    ticks_font = font_manager.FontProperties(family='Computer Modern Roman', style='normal',
                                             size=font_size, weight='normal', stretch='normal')
    rc('text', usetex=True)
    rc('font', **font_properties)

    def name_length(name):
        length_converter = 2
        return len(name) / length_converter + names_lines_space

    # figure dimensions
    x_min, x_max = inf, -inf
    for i, [alg_description, alg_rank] in enumerate(sorted_algos_copy):
        m = alg_rank + (2 * int(i >= n // 2) - 1) * name_length(alg_description)
        x_max = max(x_max, m)
        x_min = min(x_min, m)
    x_left = x_min
    x_right = x_max
    x_min = min(x_min, 1)
    x_max = max(x_max, n)

    y_min = -1
    y_max = first_level_height + inter_lines_space * (1 + n//2)
    absolute_width, absolute_height = 16, 0.5 * n
    plt.rcParams['figure.figsize'] = absolute_width, max(absolute_height, 5)
    left_bonus, right_bonus = center(absolute_width, n)
    # plotting
    fig = plt.figure(figsize=(6,5))
    ax = fig.add_subplot(111, autoscale_on=False,
                         xlim=(x_min - 0.2 - left_bonus, x_max + 0.2 + right_bonus),
                         ylim=(y_min, max(y_max, 3)))
    

    def plot_algorithm(algorithm_index, algorithm, avg_rank):
        if algorithm_index < n // 2:
            # go left
            sign = -1
            offset = 0
            alignment = 'left'
            x_end_of_line = x_left + end_of_line_manipulator
        else:
            sign = 1
            offset = n // 2
            alignment = 'right'
            x_end_of_line = x_right - end_of_line_manipulator
        line_xs = [avg_rank, avg_rank, x_end_of_line]
        height = (algorithm_index + 1 - offset) * inter_lines_space + first_level_height
        line_ys = [0, height, height]
        plt.plot(line_xs, line_ys, 'k', linewidth=1.2)
        colour = 'k' if the_algorithm_candidate != algorithm else 'b'  # index does not work here ...
        text_x = x_end_of_line - sign * names_lines_space
        ax.text(text_x, height + names_lines_space, algorithm,
                horizontalalignment=alignment,
                verticalalignment='center',
                color=colour,
                fontsize=font_size)

    def plot_critical_distance():
        y = critical_distance_offset
        x0 = 1
        if draw_stats:
            if draw_cd_line:
                plt.plot([x0, critical_distance + x0], [y, y], '|r', markersize=12, markeredgecolor='r', markeredgewidth=2)
                plt.plot([x0, critical_distance + x0], [y, y], 'r', linewidth=2)
            if show_cd_text:
                ax.text(x0, y + names_lines_space, "{}: {:.4f}".format("critical distance", critical_distance),
                        horizontalalignment='left',
                        color='r',
                        fontsize=font_size)

    def algorithm_groups():
        sorted_ranks = [t[1] for t in sorted_algorithms]  # only ranks
        intervals = set()
        for start in range(len(sorted_ranks)):
            for end in range(start + 1, len(sorted_ranks)):
                if sorted_ranks[end] - sorted_ranks[start] < critical_distance:
                    if the_index is None:
                        intervals.add((start, end))
                    elif start == the_index or end == the_index:
                        intervals.add((start, end))
        found_anything = True
        while found_anything:
            found_anything = False
            unnecessary_intervals = []
            for a in intervals:
                for b in intervals:
                    if a != b and a[0] <= b[0] < b[1] <= a[1]:
                        unnecessary_intervals.append(b)
                        found_anything = True
            for unnecessary_interval in unnecessary_intervals:
                intervals -= {unnecessary_interval}
        groups = sorted(intervals, key=lambda t: t[0])
        if the_index is not None and groups:
            # only one 'interval'
            groups = [(groups[0][0], groups[-1][1])]
        return groups

    def plot_groups(intervals):
        if draw_stats:
            k = len(intervals)
            start, end = 0, inter_lines_space + first_level_height
            heights = [start * (1 - t / (k + 1)) + end * t/(k + 1) for t in range(1, k + 1)]
            colours = ['|r', 'r'] if the_index is None else ['|b', 'b']
            for ind, [ind1, ind2] in enumerate(intervals):
                y = heights[ind]
                start = sorted_algorithms[ind1][1] - min(deltas[ind1], link_length_bonus)
                end = sorted_algorithms[ind2][1] + min(deltas[ind2 + 1], link_length_bonus)
                plt.plot([start, end], [y, y], colours[0], markersize=12, markeredgecolor=colours[1], markeredgewidth=1.5)
                plt.plot([start, end], [y, y], colours[1], linewidth=1)
        
    # hide 'axes box'
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['bottom'].set_color('none')
    # change ticks
    plt.tick_params(
        axis='y',          # changes apply to the alg_rank-axis
        which='both',      # both major and minor ticks are affected
        left=False,        # ticks along the bottom edge are off
        right=False,       # ticks along the top edge are off
        labelleft=False)   # labels along the bottom edge are off
    plt.tick_params(
        axis='x',
        which='both',
        top=False,
        bottom=False,
        )

    plt.tick_params('both', length=15, width=1, which='major')
    # line of algorithm ranks at y = 0
    ax.spines['bottom'].set_position('zero')
    # draw ticks
    plt.xticks(range(1, 1 + n), range(1, 1 + n))
    # algorithm ranks line
    plt.plot([1, n], [0, 0], 'k')
    # algorithm descriptions
    for i, alg_rank in enumerate(sorted_algos_copy):
        plot_algorithm(i, alg_rank[0], alg_rank[1])
    # critical distance
    plot_critical_distance()
    # algorithm groups
    plot_groups(algorithm_groups())
    # save / show the results
    if output_figure_file is not None:
        folder_end = output_figure_file.rfind("/")
        if folder_end >= 0:
            fig_folder = output_figure_file[:folder_end]
            if not exists(fig_folder):
                makedirs(fig_folder)
        fig.savefig(output_figure_file, bbox_inches='tight', pad_inches=0)
        plt.clf()
        print("Plot saved to", output_figure_file)
    else:
        plt.show()

    return output_figure_file


def remove_backslash(file_name):
    ch_list = []
    for ch in file_name:
        if ch != "\\":
            ch_list.append(ch)
        else:
            ch_list.append("/")
    return "".join(ch_list)

In [575]:
# Example performance data across 3 datasets
performance_data = [
    [('Alg1', 0.75), ('Alg2', 0.65), ('Alg3', 0.80)],  # Dataset 1
    [('Alg1', 0.70), ('Alg2', 0.60), ('Alg3', 0.85)],  # Dataset 2
    [('Alg1', 0.80), ('Alg2', 0.70), ('Alg3', 0.78)],  # Dataset 3
]

In [576]:
import pickle
import os

performance_data = {}

for dataset in os.listdir('../final_results/str/'):
  if not dataset.endswith('.pcl'):
     continue
  performance_data[dataset.split('.')[0]] = []
  with open(f'../final_results/str/{dataset}', 'rb') as f:
      print(dataset)
      data = pickle.load(f)
      performance_data[dataset.split('.')[0]].append(data)

era.pcl
cpmp2015.pcl
qsar234.pcl
puma8NH.pcl
yprop.pcl
space_ga.pcl
bike_sharing.pcl
quake.pcl
concrete_compressive_strength.pcl
ailerons.pcl


In [577]:
dct = {}
for folder in os.listdir('../../../../clus/data/str/'):
  dct[folder] = []
  for file in os.listdir(os.path.join('../../../../clus/data/str/',folder)):
    if not file.endswith('.pcl'): continue
    with open(f'../../../../clus/data/str/{folder}/{file}', 'rb') as f:
      data = pickle.load(f)
      dct[folder].append(data)
dct

{'quake': [('PCT-Ensemble', 0.14774334862385322),
  ('OPCT', 0.17235000000000006)],
 'qsar234': [('PCT-Ensemble', 0.4287214452214452),
  ('OPCT', 0.44101958041958045)],
 'space_ga': [('PCT-Ensemble', 0.08479289500861019),
  ('OPCT', 0.09938741832192995)],
 'era': [('PCT-Ensemble', 1.2776215), ('OPCT', 1.2774235000000003)],
 'yprop': [('PCT-Ensemble', 0.019681947664603263),
  ('OPCT', 0.0236869617332583)],
 'puma8NH': [],
 'ailerons': [('PCT-Ensemble', 0.0001330909090909091),
  ('OPCT', 0.00016112727272727274)],
 'concrete_compressive_strength': [('PCT-Ensemble', 4.164037988427184),
  ('OPCT', 4.216045586390175)],
 'cpmp2015': [('PCT-Ensemble', 1211.1244092417062),
  ('OPCT', 848.7213196682465)],
 'bike_sharing': [('PCT-Ensemble', 10.610926783659378),
  ('OPCT', 2.509828883774454)]}

In [578]:
for k,v in performance_data.items():
  performance_data[k] = []
  for item in v:
    for it in item:
      performance_data[k].append(it)
  for item in dct[k]:
    performance_data[k].append(item)

In [579]:
performance_data

{'era': [('vspyct', 1.2878398197889327),
  ('spyct_single', 1.296625337600708),
  ('spyct_ensemble', 1.2493518805503845),
  ('PCT-Ensemble', 1.2776215),
  ('OPCT', 1.2774235000000003)],
 'cpmp2015': [('vspyct', 285.10579916199237),
  ('spyct_single', 384.377425207346),
  ('spyct_ensemble', 326.4975145529797),
  ('PCT-Ensemble', 1211.1244092417062),
  ('OPCT', 848.7213196682465)],
 'qsar234': [('vspyct', 0.4287244212099444),
  ('spyct_single', 0.5725774961707197),
  ('spyct_ensemble', 0.4156559689395078),
  ('PCT-Ensemble', 0.4287214452214452),
  ('OPCT', 0.44101958041958045)],
 'puma8NH': [('vspyct', 2.7116777454818934),
  ('spyct_single', 2.795463140270826),
  ('spyct_ensemble', 2.6322416518570284)],
 'yprop': [('vspyct', 0.01996911274779561),
  ('spyct_single', 0.020125647681273298),
  ('spyct_ensemble', 0.01969259618392927),
  ('PCT-Ensemble', 0.019681947664603263),
  ('OPCT', 0.0236869617332583)],
 'space_ga': [('vspyct', 0.10450143841367766),
  ('spyct_single', 0.10571591132177637

In [580]:
import pickle
import os

with open(f'../final_results/str/ailerons.pcl', 'rb') as f:
    data = pickle.load(f)
data

[('vspyct', 0.00012723311051966794),
 ('spyct_single', 0.00013546250891030174),
 ('spyct_ensemble', 0.00012414695714945822)]

In [581]:
performance_data = list(performance_data.values())

In [582]:
performance_data_renamed = []
for el in performance_data:
    empty_list = []
    for k,v in el:
        if k=='vspyct': empty_list.append(('VSPYCT',v))
        elif k=='spyct_single': empty_list.append(('SPYCT-Single',v))
        elif k=='spyct_ensemble': empty_list.append(('SPYCT-Ensemble',v))
        elif k=='O-PCT': empty_list.append(('OPCT',v))
        else: empty_list.append((k,v))
    performance_data_renamed.append(empty_list)

In [583]:
performance_data

[[('vspyct', 1.2878398197889327),
  ('spyct_single', 1.296625337600708),
  ('spyct_ensemble', 1.2493518805503845),
  ('PCT-Ensemble', 1.2776215),
  ('OPCT', 1.2774235000000003)],
 [('vspyct', 285.10579916199237),
  ('spyct_single', 384.377425207346),
  ('spyct_ensemble', 326.4975145529797),
  ('PCT-Ensemble', 1211.1244092417062),
  ('OPCT', 848.7213196682465)],
 [('vspyct', 0.4287244212099444),
  ('spyct_single', 0.5725774961707197),
  ('spyct_ensemble', 0.4156559689395078),
  ('PCT-Ensemble', 0.4287214452214452),
  ('OPCT', 0.44101958041958045)],
 [('vspyct', 2.7116777454818934),
  ('spyct_single', 2.795463140270826),
  ('spyct_ensemble', 2.6322416518570284)],
 [('vspyct', 0.01996911274779561),
  ('spyct_single', 0.020125647681273298),
  ('spyct_ensemble', 0.01969259618392927),
  ('PCT-Ensemble', 0.019681947664603263),
  ('OPCT', 0.0236869617332583)],
 [('vspyct', 0.10450143841367766),
  ('spyct_single', 0.10571591132177637),
  ('spyct_ensemble', 0.0901931816581314),
  ('PCT-Ensemble'

In [584]:
performance_data_renamed

[[('VSPYCT', 1.2878398197889327),
  ('SPYCT-Single', 1.296625337600708),
  ('SPYCT-Ensemble', 1.2493518805503845),
  ('PCT-Ensemble', 1.2776215),
  ('OPCT', 1.2774235000000003)],
 [('VSPYCT', 285.10579916199237),
  ('SPYCT-Single', 384.377425207346),
  ('SPYCT-Ensemble', 326.4975145529797),
  ('PCT-Ensemble', 1211.1244092417062),
  ('OPCT', 848.7213196682465)],
 [('VSPYCT', 0.4287244212099444),
  ('SPYCT-Single', 0.5725774961707197),
  ('SPYCT-Ensemble', 0.4156559689395078),
  ('PCT-Ensemble', 0.4287214452214452),
  ('OPCT', 0.44101958041958045)],
 [('VSPYCT', 2.7116777454818934),
  ('SPYCT-Single', 2.795463140270826),
  ('SPYCT-Ensemble', 2.6322416518570284)],
 [('VSPYCT', 0.01996911274779561),
  ('SPYCT-Single', 0.020125647681273298),
  ('SPYCT-Ensemble', 0.01969259618392927),
  ('PCT-Ensemble', 0.019681947664603263),
  ('OPCT', 0.0236869617332583)],
 [('VSPYCT', 0.10450143841367766),
  ('SPYCT-Single', 0.10571591132177637),
  ('SPYCT-Ensemble', 0.0901931816581314),
  ('PCT-Ensemble'

In [585]:
import numpy as np
from statsmodels.stats.libqsturng import qsturng


# Step 2: Compute ranks for each dataset
ranks = {alg_name: [] for alg_name, _ in performance_data_renamed[0]}

for dataset in performance_data_renamed:
    sorted_algos = sorted(dataset, key=lambda x: x[1])  # Assuming lower score is better
    for rank, (alg_name, _) in enumerate(sorted_algos, start=1):
        ranks[alg_name].append(rank)

# Step 3: Calculate average ranks
avg_ranks = [(alg_name, np.mean(rank_list)) for alg_name, rank_list in ranks.items()]

# Step 4: Calculate the critical distance
k = len(avg_ranks)
N = len(performance_data_renamed)
alpha = 0.05

q_alpha = qsturng(1 - alpha, k, np.inf)
critical_distance = q_alpha * np.sqrt(k * (k + 1) / (6 * N))

In [586]:
# Step 5: Prepare parameters
avg_ranks_sorted = sorted(avg_ranks, key=lambda x: x[1])
list_of_algorithms = avg_ranks_sorted
the_algorithm_candidate = None
output_figure_file = '../reports/figures/critical_distance_str.pdf'
draw_cd_line = False
show_cd_text = False
draw_stats = False

In [587]:
# Step 6: Call the diagram function
diagram(
    list_of_algorithms,
    critical_distance,
    the_algorithm_candidate,
    output_figure_file,
    draw_cd_line,
    show_cd_text,
    draw_stats
)

Plot saved to ../reports/figures/test.pdf


'../reports/figures/test.pdf'

<Figure size 600x500 with 0 Axes>